## GBDT+LR

### Description
- 训练三个模型对数据进行预测
- 分别是LR模型、GBDT模型和两者的组合模型
- GBDT负责对各个特征进行交叉，把原始特征向量转换为新的离散型特征向量，然后再使用逻辑回归模型
- 然后分别观察它们的预测效果

特征的处理方式：
- 逻辑回归模型：连续特征归一化， 离散特征one-hot编码
- GBDT模型：离散特征one-hot编码（树模型连续特征不需要归一化）
- GBDT+LR模型：离散特征one-hot编码（由于LR使用的特征是GBDT的输出， 原数据依然是GBDT进行处理， 所以只需要离散特征one-hot编码）

任务： 
- 开发预测广告点击率(CTR)的模型
- 即给定一个用户和他正在访问的页面，预测他点击给定广告的概率

Dataset：
- Label：目标变量，0表示未点击， 1表示点击
- l1-l13: 13列的数值特征，大部分是计数特征
- C1-C26: 26列的分类特征，这些特征的值离散成了32位的数据

### import modules

In [1]:
# import modules

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import log_loss
import gc
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')

### import dataset

In [2]:
# import data set

path = '/Users/quzihanwu/Desktop/GBDT+LR建模/TSG_RecommenderSystem/data/'
df_train = pd.read_csv(path + 'train.csv')
df_test = pd.read_csv(path + 'test.csv')

In [3]:
df_train

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,10000743,1,1.0,0,1.0,NaN,227.0,1.0,173.0,18.0,...,3486227d,e88ffc9d,c393dc22,b1252a9d,57c90cd9,NaN,bcdee96c,4d19a3eb,cb079c2d,456c12a0
1,10000159,1,4.0,1,1.0,2.0,27.0,2.0,4.0,2.0,...,07c540c4,92555263,NaN,NaN,242bb710,NaN,3a171ecb,72c78f11,NaN,NaN
2,10001166,1,0.0,806,NaN,NaN,1752.0,142.0,2.0,0.0,...,07c540c4,25c88e42,21ddcdc9,b1252a9d,a0136dd2,NaN,32c7478e,8fc66e78,001f3601,f37f3967
3,10000318,0,2.0,-1,42.0,14.0,302.0,38.0,25.0,38.0,...,e5ba7672,5aed7436,21ddcdc9,b1252a9d,c3abeb21,NaN,423fab69,1793a828,e8b83407,5cef228f
4,10000924,1,0.0,57,2.0,1.0,2891.0,2.0,35.0,1.0,...,e5ba7672,642f2610,1d1eb838,b1252a9d,1640d50b,ad3062eb,423fab69,45ab94c8,2bf691b1,c84c4aec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,10000835,0,NaN,8,1.0,1.0,43.0,NaN,0.0,1.0,...,1e88c74f,fc35e8fe,NaN,NaN,a02708ad,c9d4222a,c3dc6cef,502f2493,NaN,NaN
1595,10001216,0,8.0,2,20.0,8.0,36.0,9.0,8.0,10.0,...,e5ba7672,5aed7436,21ddcdc9,b1252a9d,eea796be,NaN,3a171ecb,1793a828,e8b83407,5cef228f
1596,10001653,0,0.0,1,2.0,12.0,4877.0,140.0,13.0,34.0,...,e5ba7672,2b0a9d11,NaN,NaN,7453e535,NaN,dbb486d7,906e72ec,NaN,NaN
1597,10000559,0,NaN,2,NaN,1.0,1972.0,NaN,0.0,1.0,...,e5ba7672,817481a8,NaN,NaN,e4244d7f,c9d4222a,c7dc6720,60efe6e6,NaN,NaN


In [4]:
df_test

,Id,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,10000405,NaN,-1,NaN,NaN,8020.0,26.0,6.0,0.0,80.0,...,e5ba7672,7119e567,1d04f4a4,b1252a9d,d5f54153,NaN,32c7478e,a9d771cd,c9f3bea7,0a47000d
1,10001189,NaN,-1,NaN,NaN,17881.0,9.0,8.0,0.0,0.0,...,e5ba7672,51369abb,NaN,NaN,d4b6b7e8,NaN,32c7478e,37821b83,NaN,NaN
2,10000674,0.0,0,2.0,13.0,2904.0,104.0,1.0,3.0,100.0,...,e5ba7672,bd17c3da,966f1c31,a458ea53,1d1393f4,ad3062eb,32c7478e,3fdb382b,010f6491,49d68486
3,10001358,0.0,1471,51.0,4.0,1573.0,63.0,1.0,4.0,13.0,...,d4bb7bd8,1f9656b8,21ddcdc9,b1252a9d,602ce342,NaN,3a171ecb,1793a828,e8b83407,70b6702c
4,10000810,0.0,16,9.0,17.0,2972.0,621.0,13.0,42.0,564.0,...,e5ba7672,87c6f83c,NaN,NaN,bf8efd4c,c9d4222a,423fab69,f96a556f,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,10001453,1.0,0,1.0,NaN,149.0,5.0,1.0,0.0,0.0,...,d4bb7bd8,5aed7436,d16737e3,a458ea53,edc49a33,NaN,93bad2c0,3fdb382b,e8b83407,80dd0a5b
396,10000360,NaN,-1,NaN,NaN,NaN,NaN,0.0,0.0,6.0,...,2005abd1,5162930e,NaN,NaN,12965bb8,NaN,32c7478e,71292dbb,NaN,NaN
397,10001809,0.0,300,4.0,NaN,4622.0,25.0,20.0,6.0,55.0,...,8efede7f,a78bd508,21ddcdc9,5840adea,c2a93b37,NaN,3a171ecb,1793a828,e8b83407,2fede552
398,10000769,1.0,1,2.0,1.0,5.0,1.0,1.0,1.0,1.0,...,d4bb7bd8,a1d0cc4f,c68db44a,a458ea53,3b1ae854,NaN,32c7478e,57e2c6c9,1575c75f,7132fed8


### data Preprocessing

In [5]:
# data preprocessing

# 去掉 Id 列
df_train.drop(['Id'], axis=1, inplace=True)
df_test.drop(['Id'], axis=1, inplace=True)

# 测试集新增 Label 列
df_test['Label'] = -1

# 合并测试集和训练集
data = pd.concat([df_train, df_test])

# 填充缺失值
data.fillna(-1, inplace=True)

In [6]:
data

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1,1.0,0,1.0,-1.0,227.0,1.0,173.0,18.0,50.0,...,3486227d,e88ffc9d,c393dc22,b1252a9d,57c90cd9,-1,bcdee96c,4d19a3eb,cb079c2d,456c12a0
1,1,4.0,1,1.0,2.0,27.0,2.0,4.0,2.0,2.0,...,07c540c4,92555263,-1,-1,242bb710,-1,3a171ecb,72c78f11,-1,-1
2,1,0.0,806,-1.0,-1.0,1752.0,142.0,2.0,0.0,50.0,...,07c540c4,25c88e42,21ddcdc9,b1252a9d,a0136dd2,-1,32c7478e,8fc66e78,001f3601,f37f3967
3,0,2.0,-1,42.0,14.0,302.0,38.0,25.0,38.0,90.0,...,e5ba7672,5aed7436,21ddcdc9,b1252a9d,c3abeb21,-1,423fab69,1793a828,e8b83407,5cef228f
4,1,0.0,57,2.0,1.0,2891.0,2.0,35.0,1.0,137.0,...,e5ba7672,642f2610,1d1eb838,b1252a9d,1640d50b,ad3062eb,423fab69,45ab94c8,2bf691b1,c84c4aec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,-1,1.0,0,1.0,-1.0,149.0,5.0,1.0,0.0,0.0,...,d4bb7bd8,5aed7436,d16737e3,a458ea53,edc49a33,-1,93bad2c0,3fdb382b,e8b83407,80dd0a5b
396,-1,-1.0,-1,-1.0,-1.0,-1.0,-1.0,0.0,0.0,6.0,...,2005abd1,5162930e,-1,-1,12965bb8,-1,32c7478e,71292dbb,-1,-1
397,-1,0.0,300,4.0,-1.0,4622.0,25.0,20.0,6.0,55.0,...,8efede7f,a78bd508,21ddcdc9,5840adea,c2a93b37,-1,3a171ecb,1793a828,e8b83407,2fede552
398,-1,1.0,1,2.0,1.0,5.0,1.0,1.0,1.0,1.0,...,d4bb7bd8,a1d0cc4f,c68db44a,a458ea53,3b1ae854,-1,32c7478e,57e2c6c9,1575c75f,7132fed8


In [7]:
# 划分连续和离散特征

continuous_fea = ['I'+str(i+1) for i in range(13)]
category_fea = ['C'+str(i+1) for i in range(26)]

print(data[: 3])

   Label   I1   I2   I3   I4      I5     I6     I7    I8    I9  ...       C17  \
0      1  1.0    0  1.0 -1.0   227.0    1.0  173.0  18.0  50.0  ...  3486227d   
1      1  4.0    1  1.0  2.0    27.0    2.0    4.0   2.0   2.0  ...  07c540c4   
2      1  0.0  806 -1.0 -1.0  1752.0  142.0    2.0   0.0  50.0  ...  07c540c4   

        C18       C19       C20       C21 C22       C23       C24       C25  \
0  e88ffc9d  c393dc22  b1252a9d  57c90cd9  -1  bcdee96c  4d19a3eb  cb079c2d   
1  92555263        -1        -1  242bb710  -1  3a171ecb  72c78f11        -1   
2  25c88e42  21ddcdc9  b1252a9d  a0136dd2  -1  32c7478e  8fc66e78  001f3601   

        C26  
0  456c12a0  
1        -1  
2  f37f3967  

[3 rows x 40 columns]


### LR mode

In [8]:
# LR
def lr_model(data, category_fea, continuous_fea):
    """
        逻辑回归建模
        
        :param data: df, 数据表
        :param category_fea: list, 离散特征列
        :param continuous_fea: list, 连续特征列
    """
    
    # 连续特征归一化
    scaler = MinMaxScaler()
    for col in continuous_fea:
        data[col] = scaler.fit_transform(data[col].values.reshape(-1, 1))
    
    # 离散特征one-hot编码
    for col in category_fea:
        onehot_feats = pd.get_dummies(data[col], prefix=col)
        data.drop([col], axis=1, inplace=True)
        data = pd.concat([data, onehot_feats], axis=1)
    
    # 拆分训练集和测试集
    train = data[data['Label'] != -1]
    target = train.pop('Label')
    test = data[data['Label'] == -1]
    test.drop(['Label'], axis=1, inplace=True)
    
    # 划分训练集
    x_train, x_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=2020)

    # 建立模型
    lr = LogisticRegression()
    lr.fit(x_train, y_train)
    tr_logloss = log_loss(y_train, lr.predict_proba(x_train)[:, 1])  # −(ylog(p)+(1−y)log(1−p)) log_loss
    val_logloss = log_loss(y_val, lr.predict_proba(x_val)[:, 1])
    print('tr_logloss: ', tr_logloss)
    print('val_logloss: ', val_logloss)
    
    # 模型预测
    y_pred = lr.predict_proba(test)[:, 1]  # predict_proba 返回n行k列的矩阵，第i行第j列上的数值是模型预测第i个预测样本为某个标签的概率，这里的1表示点击的概率
    print('predict: ', y_pred[:10]) # 这里看前10个，预测为点击的概率

In [9]:
lr_model(data.copy(), category_fea, continuous_fea)

tr_logloss:  0.1242339516478198
val_logloss:  0.44407245698797837
predict:  [0.44783059 0.80628705 0.1756691  0.02070154 0.13984202 0.46490042
 0.43386417 0.07089967 0.07121148 0.27896238]


### GBDT model

In [10]:
# GBDT

def gbdt_model(data, category_fea, continuous_fea):
    """
        GBDT建模
        
        :param data: df, 数据表
        :param category_fea: list, 离散特征列
        :param continuous_fea: list, 连续特征列
    """
    
    # 离散特征one-hot编码
    for col in category_fea:
        onehot_feats = pd.get_dummies(data[col], prefix=col)
        data.drop([col], axis=1, inplace=True)
        data = pd.concat([data, onehot_feats], axis=1)

    # 拆分训练集和测试集
    train = data[data['Label'] != -1]
    target = train.pop('Label')
    test = data[data['Label'] == -1]
    test.drop(['Label'], axis=1, inplace=True)
    
    # 划分数据集
    x_train, x_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=2020)
    
    # 建立模型
    gbm = lgb.LGBMClassifier(boosting_type='gbdt',  # 这里用gbdt
                             objective='binary', 
                             subsample=0.8,
                             min_child_weight=0.5, 
                             colsample_bytree=0.7,
                             num_leaves=100,
                             max_depth=12,
                             learning_rate=0.01,
                             n_estimators=10000
                            )
    gbm.fit(x_train, y_train, 
            eval_set=[(x_train, y_train), (x_val, y_val)], 
            eval_names=['train', 'val'],
            eval_metric='binary_logloss',
            early_stopping_rounds=100,
           )
    
    tr_logloss = log_loss(y_train, gbm.predict_proba(x_train)[:, 1])   # −(ylog(p)+(1−y)log(1−p)) log_loss
    val_logloss = log_loss(y_val, gbm.predict_proba(x_val)[:, 1])
    print('tr_logloss: ', tr_logloss)
    print('val_logloss: ', val_logloss)
    
    # 模型预测
    y_pred = gbm.predict_proba(test)[:, 1]  # predict_proba 返回n行k列的矩阵，第i行第j列上的数值是模型预测第i个预测样本为某个标签的概率，这里的1表示点击的概率
    print('predict: ', y_pred[:10]) # 这里看前10个，预测为点击的概率


In [11]:
gbdt_model(data.copy(), category_fea, continuous_fea)

[1]	train's binary_logloss: 0.523857	val's binary_logloss: 0.457806
[2]	train's binary_logloss: 0.521371	val's binary_logloss: 0.457213
[3]	train's binary_logloss: 0.519084	val's binary_logloss: 0.456616
[4]	train's binary_logloss: 0.516882	val's binary_logloss: 0.456046
[5]	train's binary_logloss: 0.514449	val's binary_logloss: 0.455649
[6]	train's binary_logloss: 0.512277	val's binary_logloss: 0.455319
[7]	train's binary_logloss: 0.509973	val's binary_logloss: 0.455039
[8]	train's binary_logloss: 0.507717	val's binary_logloss: 0.454523
[9]	train's binary_logloss: 0.505668	val's binary_logloss: 0.454546
[10]	train's binary_logloss: 0.503491	val's binary_logloss: 0.454134
[11]	train's binary_logloss: 0.501469	val's binary_logloss: 0.453151
[12]	train's binary_logloss: 0.499463	val's binary_logloss: 0.452609
[13]	train's binary_logloss: 0.497257	val's binary_logloss: 0.452419
[14]	train's binary_logloss: 0.495206	val's binary_logloss: 0.451944
[15]	train's binary_logloss: 0.493127	val's

### GBDT+LR model

In [12]:
def gbdt_lr_model(data, category_feature, continuous_feature):  # 0.43616
    """
        GBDT+LR建模
        
        :param data: df, 数据表
        :param category_fea: list, 离散特征列
        :param continuous_fea: list, 连续特征列
    """
    
    # 离散特征one-hot编码
    for col in category_feature:
        onehot_feats = pd.get_dummies(data[col], prefix = col)
        data.drop([col], axis = 1, inplace = True)
        data = pd.concat([data, onehot_feats], axis = 1)

    # 拆分训练集和测试集
    train = data[data['Label'] != -1]
    target = train.pop('Label')
    test = data[data['Label'] == -1]
    test.drop(['Label'], axis = 1, inplace = True)

    # 划分数据集
    x_train, x_val, y_train, y_val = train_test_split(train, target, test_size = 0.2, random_state = 2020)

    # 建立GBDT模型
    gbm = lgb.LGBMClassifier(objective='binary',
                            subsample= 0.8,
                            min_child_weight= 0.5,
                            colsample_bytree= 0.7,
                            num_leaves=100,
                            max_depth = 12,
                            learning_rate=0.01,
                            n_estimators=1000,
                            )

    gbm.fit(x_train, y_train,
            eval_set = [(x_train, y_train), (x_val, y_val)],
            eval_names = ['train', 'val'],
            eval_metric = 'binary_logloss',
            early_stopping_rounds = 100,
            )
    
    # GBDT负责对各个特征进行交叉， 把原始特征向量转换为新的离散型特征向量
    model = gbm.booster_

    gbdt_feats_train = model.predict(train, pred_leaf = True)
    gbdt_feats_test = model.predict(test, pred_leaf = True)
    gbdt_feats_name = ['gbdt_leaf_' + str(i) for i in range(gbdt_feats_train.shape[1])]
    df_train_gbdt_feats = pd.DataFrame(gbdt_feats_train, columns = gbdt_feats_name) 
    df_test_gbdt_feats = pd.DataFrame(gbdt_feats_test, columns = gbdt_feats_name)

    # 新特征（连续特征 + 离散特征one-hot编码 + GBDT生成的离散特征）
    train = pd.concat([train, df_train_gbdt_feats], axis = 1)
    test = pd.concat([test, df_test_gbdt_feats], axis = 1)
    train_len = train.shape[0]
    data = pd.concat([train, test])    
    del train
    del test
    gc.collect()  # 清理内存

    # 连续特征归一化
    scaler = MinMaxScaler()
    for col in continuous_feature:
        data[col] = scaler.fit_transform(data[col].values.reshape(-1, 1))

    # GBDT生成的离散特征one-hot编码
    for col in gbdt_feats_name:
        onehot_feats = pd.get_dummies(data[col], prefix = col)
        data.drop([col], axis = 1, inplace = True)
        data = pd.concat([data, onehot_feats], axis = 1)

    train = data[: train_len]
    test = data[train_len:]
    del data
    gc.collect()

    # 划分数据集
    x_train, x_val, y_train, y_val = train_test_split(train, target, test_size = 0.3, random_state = 2018)

    # 建立逻辑回归模型
    lr = LogisticRegression()
    lr.fit(x_train, y_train)
    tr_logloss = log_loss(y_train, lr.predict_proba(x_train)[:, 1])
    val_logloss = log_loss(y_val, lr.predict_proba(x_val)[:, 1])
    print('tr-logloss: ', tr_logloss)
    print('val-logloss: ', val_logloss)
    
    # 模型预测
    y_pred = lr.predict_proba(test)[:, 1]
    print(y_pred[:10])


In [13]:
gbdt_lr_model(data.copy(), category_fea, continuous_fea)

[1]	train's binary_logloss: 0.523857	val's binary_logloss: 0.457806
[2]	train's binary_logloss: 0.521371	val's binary_logloss: 0.457213
[3]	train's binary_logloss: 0.519084	val's binary_logloss: 0.456616
[4]	train's binary_logloss: 0.516882	val's binary_logloss: 0.456046
[5]	train's binary_logloss: 0.514449	val's binary_logloss: 0.455649
[6]	train's binary_logloss: 0.512277	val's binary_logloss: 0.455319
[7]	train's binary_logloss: 0.509973	val's binary_logloss: 0.455039
[8]	train's binary_logloss: 0.507717	val's binary_logloss: 0.454523
[9]	train's binary_logloss: 0.505668	val's binary_logloss: 0.454546
[10]	train's binary_logloss: 0.503491	val's binary_logloss: 0.454134
[11]	train's binary_logloss: 0.501469	val's binary_logloss: 0.453151
[12]	train's binary_logloss: 0.499463	val's binary_logloss: 0.452609
[13]	train's binary_logloss: 0.497257	val's binary_logloss: 0.452419
[14]	train's binary_logloss: 0.495206	val's binary_logloss: 0.451944
[15]	train's binary_logloss: 0.493127	val's

### Results

| Model | train's binary_logloss | val's binary_logloss |
| :---- | -----: | :----:|
| LR | 0.124234 | 0.444072 |
| GBDT | 0.330627 | 0.434013 |
| GBDT+LR | 0.012044 | 0.305277 |